# Multiple Linear Regression Model - Features Selection

In [1]:
import pandas as pd
import numpy as np
import statistics as stat
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import RidgeCV
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import random

random.seed(10)

pd.options.mode.chained_assignment = None 


ad =  pd.read_csv('dataset.csv')

### A glimpse at the data

In [2]:
ad.head()
#https://scikit-learn.org/stable/auto_examples/feature_selection/plot_select_from_model_diabetes.html#sphx-glr-auto-examples-feature-selection-plot-select-from-model-diabetes-py

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1698,0.701,1.034


In [3]:
ad.isnull().values.any()

True

### Grouping CDR 2.0 to 1.0


In [4]:
ad.loc[ad["CDR"] == 2,"CDR"] = 1

In [5]:
ad["Male"] = np.where(ad["M/F"]=="M", 1, 0)
ad["Female"] = np.where(ad["M/F"]=="F", 1, 0)

In [6]:
#ad = ad.dropna()

### Splitting the columns into indepdent and dependent variables

In [7]:
X = ad.iloc[:,[3,4,7,8,9,10,12,13,14,15,16]] #dropped 12
y = ad.iloc[:,[11]]

y=y.astype(str)


### Normalizing the values

In [8]:
def normalize(x):
    return (x-min(x))/((max(x)) - min(x))

In [9]:
X.loc[:,['Visit']]=pd.DataFrame(normalize(X['Visit']))
X.loc[:,['MR Delay']]=pd.DataFrame(normalize(X['MR Delay']))
X.loc[:,['Age']]=pd.DataFrame(normalize(X['Age']))
X.loc[:,['EDUC']]=pd.DataFrame(normalize(X['EDUC']))
X.loc[:,['SES']]=pd.DataFrame(normalize(X['SES']))
X.loc[:,['MMSE']]=pd.DataFrame(normalize(X['MMSE']))
X.loc[:,['eTIV']]=pd.DataFrame(normalize(X['eTIV']))
X.loc[:,['nWBV']]=pd.DataFrame(normalize(X['nWBV']))
X.loc[:,['ASF']]=pd.DataFrame(normalize(X['ASF']))

### Splitting into training and test sets

In [10]:
ad_X_train, ad_X_test, ad_y_train, ad_y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Impute missing values for the training set

In [11]:
med_SES = ad_X_train['SES'].mean()
avg_MMSE = ad_X_train['MMSE'].mean()

In [12]:
ad_X_train.loc[:,'SES'] = ad_X_train.loc[:,'SES'].fillna(med_SES)
ad_X_train.loc[:,'MMSE'] = ad_X_train.loc[:,'MMSE'].fillna(avg_MMSE)

In [13]:
ad_y_train = ad_y_train.astype(float)
ad_y_train.loc[:,'CDR'] = ad_y_train.loc[:,'CDR'].fillna(stat.median(ad_y_train['CDR']))
ad_y_train = ad_y_train.astype(str)


### Complete case for test set

In [14]:
ad_X_test["CDR"] = ad_y_test
ad_X_test = ad_X_test.dropna(axis = 0, how='any')

ad_y_test = ad_X_test["CDR"]
ad_X_test=ad_X_test.loc[:,ad_X_test.columns!='CDR']

### Defining the function to check the accuracy, precise, and recall scores for number of features selected for Linear Regression

In [15]:
def acc_test(cnt,fold):
    regressor = LinearRegression()
    
    kf = KFold(n_splits=fold, random_state=4, shuffle=True)
    oversample = RandomOverSampler(sampling_strategy='minority')
    #med_SES = stat.median(ad_X_train['SES'])
    #avg_MMSE = ad_X_train['MMSE'].mean()
    acc_score = []
    precise_score = []
    recall = []

    for train_index, test_index in kf.split(ad_X_train):
        X_train, X_test = ad_X_train.iloc[train_index,:], ad_X_train.iloc[test_index,:]
        y_train, y_test = ad_y_train.iloc[train_index,:], ad_y_train.iloc[test_index,:]

        X_train, y_train = oversample.fit_resample(X_train, y_train)


        # impute missing values for training set
        #X_train.loc[:,'SES'] = X_train.loc[:,'SES'].fillna(med_SES)
        #X_train.loc[:,'MMSE'] = X_train.loc[:,'MMSE'].fillna(avg_MMSE)

       # y_train = y_train.astype(float)
        #y_train.loc[:,'CDR'] = y_train.loc[:,'CDR'].fillna(stat.median(y_train['CDR']))
        #y_train = y_train.astype(str)

       

        #Complete case for test set
        #ad_X_test["CDR"] = ad_y_test
        #ad_X_test = ad_X_test.dropna(axis = 0, how='any')

        #ad_y_test = ad_X_test["CDR"]
        #ad_X_test=ad_X_test.loc[:,ad_X_test.columns!='CDR']

        ridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(X_train,y_train)

        
        if cnt < 8:
            sfs_forward = SequentialFeatureSelector(
            ridge , n_features_to_select = cnt, direction = "forward").fit(X_train,y_train)

            regressor.fit(X_train[X_train.columns[sfs_forward.get_support()]],y_train)

            y_pred = regressor.predict(ad_X_test[ad_X_test.columns[sfs_forward.get_support()]])
        else:
            regressor.fit(X_train,y_train)
            y_pred = regressor.predict(ad_X_test)
            
        new_y_pred = np.around(y_pred * 2.0) / 2.0
        new_y_pred = new_y_pred.astype(str)
        #ad_y_test = ad_y_test.astype(str)

        acc = accuracy_score(ad_y_test, new_y_pred)
        acc_score.append(acc)

        precise = precision_score(ad_y_test, new_y_pred, average='macro',zero_division=1)
        precise_score.append(precise)

        rec = recall_score(ad_y_test, new_y_pred, average = 'macro', zero_division = 1)
        recall.append(rec)

        
    return acc_score, precise_score, recall
    
    
        
        
    

In [16]:
ad_X_test.head()

,Visit,MR Delay,Age,EDUC,SES,MMSE,eTIV,nWBV,ASF,Male,Female
327,0.0,0.0,0.578947,0.470588,0.25,0.730769,0.454343,0.176166,0.398031,1,0
33,0.0,0.0,0.710526,0.470588,0.00,1.000000,0.334076,0.367876,0.523207,0,1
15,0.0,0.0,0.210526,0.352941,0.25,0.884615,0.390869,0.839378,0.462729,1,0
314,0.0,0.0,0.473684,0.705882,0.00,1.000000,0.152561,0.538860,0.753868,0,1
57,0.0,0.0,0.684211,0.352941,0.50,1.000000,0.360802,0.383420,0.493671,0,1


### Selecting One Feature

In [17]:
a5 = acc_test(1,5)
a10 = acc_test(1,10)
a15 = acc_test(1,15)
a20 = acc_test(1,20)
a25 = acc_test(1,25)

a5_acc = stat.mean(a5[0])
a10_acc = stat.mean(a10[0])
a15_acc = stat.mean(a15[0])
a20_acc = stat.mean(a20[0])
a25_acc = stat.mean(a25[0])

print("Accuracy Scores")
print(a5_acc)
print(a10_acc)
print(a15_acc)
print(a20_acc)
print(a25_acc)

acc1 = (a5_acc + a10_acc + a15_acc + a20_acc + a25_acc) / 5

print("The average of the accuracy score: ", acc1)

print("*******************************")
print("Precision Scores")
a5_prec = stat.mean(a5[1])
a10_prec = stat.mean(a10[1])
a15_prec = stat.mean(a15[1])
a20_prec = stat.mean(a20[1])
a25_prec = stat.mean(a25[1])

print(a5_prec)
print(a10_prec)
print(a15_prec)
print(a20_prec)
print(a25_prec)

pre1 =  (a5_prec + a10_prec + a15_prec + a20_prec + a25_prec) / 5

print("The average of the precision score: ", pre1)

print("*******************************")
print("Recall Scores")
a5_rec = stat.mean(a5[2])
a10_rec = stat.mean(a10[2])
a15_rec = stat.mean(a15[2])
a20_rec = stat.mean(a20[2])
a25_rec = stat.mean(a25[2])

print(a5_rec)
print(a10_rec)
print(a15_rec)
print(a20_rec)
print(a25_rec)

rec1 = (a5_rec + a10_rec + a15_rec + a20_rec + a25_rec) / 5


print("The average of the recall score: ", rec1 )

Accuracy Scores
0.5289719626168223
0.5747663551401869
0.5775700934579439
0.5799065420560747
0.5674766355140186
The average of the accuracy score:  0.5657383177570093
*******************************
Precision Scores
0.4124823875464244
0.4005501886546968
0.4147669057377049
0.40916700807664746
0.41352143359827787
The average of the precision score:  0.4100975847227503
*******************************
Recall Scores
0.6162248297841518
0.6389779805881501
0.6407938577430103
0.6332880631609445
0.6347689410401275
The average of the recall score:  0.6328107344632768


### Selecting Two Features

In [18]:
a5 = acc_test(2,5)
a10 = acc_test(2,10)
a15 = acc_test(2,15)
a20 = acc_test(2,20)
a25 = acc_test(2,25)

a5_acc = stat.mean(a5[0])
a10_acc = stat.mean(a10[0])
a15_acc = stat.mean(a15[0])
a20_acc = stat.mean(a20[0])
a25_acc = stat.mean(a25[0])

print("Accuracy Scores")
print(a5_acc)
print(a10_acc)
print(a15_acc)
print(a20_acc)
print(a25_acc)

acc2 =  (a5_acc + a10_acc + a15_acc + a20_acc + a25_acc) / 5

print("The average of the accuracy score: ", acc2)

print("*******************************")
print("Precision Scores")
a5_prec = stat.mean(a5[1])
a10_prec = stat.mean(a10[1])
a15_prec = stat.mean(a15[1])
a20_prec = stat.mean(a20[1])
a25_prec = stat.mean(a25[1])

print(a5_prec)
print(a10_prec)
print(a15_prec)
print(a20_prec)
print(a25_prec)

pre2 = (a5_prec + a10_prec + a15_prec + a20_prec + a25_prec) / 5


print("The average of the precision score: ", pre2)

print("*******************************")
print("Recall Scores")
a5_rec = stat.mean(a5[2])
a10_rec = stat.mean(a10[2])
a15_rec = stat.mean(a15[2])
a20_rec = stat.mean(a20[2])
a25_rec = stat.mean(a25[2])

print(a5_rec)
print(a10_rec)
print(a15_rec)
print(a20_rec)
print(a25_rec)

rec2 = (a5_rec + a10_rec + a15_rec + a20_rec + a25_rec) / 5

print("The average of the recall score: ", rec2 )

Accuracy Scores
0.5570093457943924
0.5261682242990654
0.5233644859813084
0.5313084112149532
0.5398130841121495
The average of the accuracy score:  0.5355327102803737
*******************************
Precision Scores
0.3842223192044743
0.36802863256210083
0.36583769618151735
0.36798030970013557
0.36597778048101115
The average of the precision score:  0.3704093476258478
*******************************
Recall Scores
0.6724453136317543
0.6504396639142401
0.6466203100101405
0.6451825293350717
0.6561720990873533
The average of the recall score:  0.654171983195712


### Selecting Three Features

In [19]:
a5 = acc_test(3,5)
a10 = acc_test(3,10)
a15 = acc_test(3,15)
a20 = acc_test(3,20)
a25 = acc_test(3,25)

a5_acc = stat.mean(a5[0])
a10_acc = stat.mean(a10[0])
a15_acc = stat.mean(a15[0])
a20_acc = stat.mean(a20[0])
a25_acc = stat.mean(a25[0])

print("Accuracy Scores")
print(a5_acc)
print(a10_acc)
print(a15_acc)
print(a20_acc)
print(a25_acc)

acc3 =  (a5_acc + a10_acc + a15_acc + a20_acc + a25_acc) / 5

print("The average of the accuracy score: ", acc3)

print("*******************************")
print("Precision Scores")
a5_prec = stat.mean(a5[1])
a10_prec = stat.mean(a10[1])
a15_prec = stat.mean(a15[1])
a20_prec = stat.mean(a20[1])
a25_prec = stat.mean(a25[1])

print(a5_prec)
print(a10_prec)
print(a15_prec)
print(a20_prec)
print(a25_prec)

pre3 = (a5_prec + a10_prec + a15_prec + a20_prec + a25_prec) / 5


print("The average of the precision score: ", pre3)

print("*******************************")
print("Recall Scores")
a5_rec = stat.mean(a5[2])
a10_rec = stat.mean(a10[2])
a15_rec = stat.mean(a15[2])
a20_rec = stat.mean(a20[2])
a25_rec = stat.mean(a25[2])

print(a5_rec)
print(a10_rec)
print(a15_rec)
print(a20_rec)
print(a25_rec)

rec3 = (a5_rec + a10_rec + a15_rec + a20_rec + a25_rec) / 5

print("The average of the recall score: ", rec3 )

Accuracy Scores
0.5738317757009346
0.5953271028037382
0.5831775700934579
0.5785046728971962
0.5839252336448598
The average of the accuracy score:  0.5829532710280374
*******************************
Precision Scores
0.3812813866059342
0.4027061337759512
0.40314273876102263
0.39072715562304533
0.390982370464904
The average of the precision score:  0.3937679570461715
*******************************
Recall Scores
0.672617702448211
0.6831580472258438
0.6661451542807475
0.6723703462262784
0.6769253947559032
The average of the recall score:  0.6742433289873968


### Selecting Four Features

In [20]:
a5 = acc_test(4,5)
a10 = acc_test(4,10)
a15 = acc_test(4,15)
a20 = acc_test(4,20)
a25 = acc_test(4,25)

a5_acc = stat.mean(a5[0])
a10_acc = stat.mean(a10[0])
a15_acc = stat.mean(a15[0])
a20_acc = stat.mean(a20[0])
a25_acc = stat.mean(a25[0])

print("Accuracy Scores")
print(a5_acc)
print(a10_acc)
print(a15_acc)
print(a20_acc)
print(a25_acc)

acc4 =  (a5_acc + a10_acc + a15_acc + a20_acc + a25_acc) / 5

print("The average of the accuracy score: ", acc4)

print("*******************************")
print("Precision Scores")
a5_prec = stat.mean(a5[1])
a10_prec = stat.mean(a10[1])
a15_prec = stat.mean(a15[1])
a20_prec = stat.mean(a20[1])
a25_prec = stat.mean(a25[1])

print(a5_prec)
print(a10_prec)
print(a15_prec)
print(a20_prec)
print(a25_prec)

pre4 = (a5_prec + a10_prec + a15_prec + a20_prec + a25_prec) / 5
print("The average of the precision score: ", pre4)

print("*******************************")
print("Recall Scores")
a5_rec = stat.mean(a5[2])
a10_rec = stat.mean(a10[2])
a15_rec = stat.mean(a15[2])
a20_rec = stat.mean(a20[2])
a25_rec = stat.mean(a25[2])

print(a5_rec)
print(a10_rec)
print(a15_rec)
print(a20_rec)
print(a25_rec)

rec4 = (a5_rec + a10_rec + a15_rec + a20_rec + a25_rec) / 5

print("The average of the recall score: ", rec4)

Accuracy Scores
0.5607476635514018
0.5607476635514018
0.5937694704049844
0.5663551401869158
0.5970093457943925
The average of the accuracy score:  0.5757258566978194
*******************************
Precision Scores
0.3532095906457995
0.37677939130343535
0.41363744005429803
0.3779111656657944
0.41136232956927044
The average of the precision score:  0.3865799834477196
*******************************
Recall Scores
0.6969563957699552
0.6751231348688975
0.6680081124148921
0.670445458496306
0.682431986093003
The average of the recall score:  0.6785930175286108


### Selecting Five Features

In [21]:
a5 = acc_test(5,5)
a10 = acc_test(5,10)
a15 = acc_test(5,15)
a20 = acc_test(5,20)
a25 = acc_test(5,25)

a5_acc = stat.mean(a5[0])
a10_acc = stat.mean(a10[0])
a15_acc = stat.mean(a15[0])
a20_acc = stat.mean(a20[0])
a25_acc = stat.mean(a25[0])

print("Accuracy Scores")
print(a5_acc)
print(a10_acc)
print(a15_acc)
print(a20_acc)
print(a25_acc)

acc5 = (a5_acc + a10_acc + a15_acc + a20_acc + a25_acc) / 5

print("The average of the accuracy score: ", acc5)

print("*******************************")
print("Precision Scores")
a5_prec = stat.mean(a5[1])
a10_prec = stat.mean(a10[1])
a15_prec = stat.mean(a15[1])
a20_prec = stat.mean(a20[1])
a25_prec = stat.mean(a25[1])

print(a5_prec)
print(a10_prec)
print(a15_prec)
print(a20_prec)
print(a25_prec)

pre5 = (a5_prec + a10_prec + a15_prec + a20_prec + a25_prec) / 5
print("The average of the precision score: ", pre5)

print("*******************************")
print("Recall Scores")
a5_rec = stat.mean(a5[2])
a10_rec = stat.mean(a10[2])
a15_rec = stat.mean(a15[2])
a20_rec = stat.mean(a20[2])
a25_rec = stat.mean(a25[2])

print(a5_rec)
print(a10_rec)
print(a15_rec)
print(a20_rec)
print(a25_rec)

rec5 = (a5_rec + a10_rec + a15_rec + a20_rec + a25_rec) / 5
print("The average of the recall score: ", rec5)

Accuracy Scores
0.5570093457943925
0.5598130841121495
0.5806853582554516
0.5766355140186915
0.5629906542056075
The average of the accuracy score:  0.5674267912772585
*******************************
Precision Scores
0.3242384309515174
0.3743814487530344
0.35753533572487534
0.3575107905664993
0.3577841905812691
The average of the precision score:  0.3542900393154391
*******************************
Recall Scores
0.7010053599884108
0.6835817760394032
0.6975595151866338
0.6940038147665266
0.6859310444734174
The average of the recall score:  0.6924163020908785


### Selecting Six Features

In [22]:
a5 = acc_test(6,5)
a10 = acc_test(6,10)
a15 = acc_test(6,15)
a20 = acc_test(6,20)
a25 = acc_test(6,25)

a5_acc = stat.mean(a5[0])
a10_acc = stat.mean(a10[0])
a15_acc = stat.mean(a15[0])
a20_acc = stat.mean(a20[0])
a25_acc = stat.mean(a25[0])

print("Accuracy Scores")
print(a5_acc)
print(a10_acc)
print(a15_acc)
print(a20_acc)
print(a25_acc)

acc6 = (a5_acc + a10_acc + a15_acc + a20_acc + a25_acc) / 5

print("The average of the accuracy score: ", acc6)

print("*******************************")
print("Precision Scores")
a5_prec = stat.mean(a5[1])
a10_prec = stat.mean(a10[1])
a15_prec = stat.mean(a15[1])
a20_prec = stat.mean(a20[1])
a25_prec = stat.mean(a25[1])

print(a5_prec)
print(a10_prec)
print(a15_prec)
print(a20_prec)
print(a25_prec)

pre6 = (a5_prec + a10_prec + a15_prec + a20_prec + a25_prec) / 5

print("The average of the precision score: ", pre6)

print("*******************************")
print("Recall Scores")
a5_rec = stat.mean(a5[2])
a10_rec = stat.mean(a10[2])
a15_rec = stat.mean(a15[2])
a20_rec = stat.mean(a20[2])
a25_rec = stat.mean(a25[2])

print(a5_rec)
print(a10_rec)
print(a15_rec)
print(a20_rec)
print(a25_rec)

rec6 = (a5_rec + a10_rec + a15_rec + a20_rec + a25_rec) / 5
print("The average of the recall score: ", rec6)

Accuracy Scores
0.5831775700934579
0.5523364485981308
0.5725856697819314
0.5714953271028037
0.5674766355140186
The average of the accuracy score:  0.5694143302180684
*******************************
Precision Scores
0.3544627855989616
0.3381802619909944
0.36484767235729365
0.3495728263464958
0.35121637307064646
The average of the precision score:  0.35165598387287833
*******************************
Recall Scores
0.7047312762567
0.6970534550195567
0.6909474141677532
0.697824496595683
0.6945946206963156
The average of the recall score:  0.6970302525472016


### Selecting Seven Features

In [23]:
a5 = acc_test(7,5)
a10 = acc_test(7,10)
a15 = acc_test(7,15)
a20 = acc_test(7,20)
a25 = acc_test(7,25)

a5_acc = stat.mean(a5[0])
a10_acc = stat.mean(a10[0])
a15_acc = stat.mean(a15[0])
a20_acc = stat.mean(a20[0])
a25_acc = stat.mean(a25[0])

print("Accuracy Scores")
print(a5_acc)
print(a10_acc)
print(a15_acc)
print(a20_acc)
print(a25_acc)

acc7 = (a5_acc + a10_acc + a15_acc + a20_acc + a25_acc) / 5

print("The average of the accuracy score: ", acc7 )

print("*******************************")
print("Precision Scores")
a5_prec = stat.mean(a5[1])
a10_prec = stat.mean(a10[1])
a15_prec = stat.mean(a15[1])
a20_prec = stat.mean(a20[1])
a25_prec = stat.mean(a25[1])

print(a5_prec)
print(a10_prec)
print(a15_prec)
print(a20_prec)
print(a25_prec)

pre7 = (a5_prec + a10_prec + a15_prec + a20_prec + a25_prec) / 5

print("The average of the precision score: ", pre7)

print("*******************************")
print("Recall Scores")
a5_rec = stat.mean(a5[2])
a10_rec = stat.mean(a10[2])
a15_rec = stat.mean(a15[2])
a20_rec = stat.mean(a20[2])
a25_rec = stat.mean(a25[2])

print(a5_rec)
print(a10_rec)
print(a15_rec)
print(a20_rec)
print(a25_rec)

rec7 = (a5_rec + a10_rec + a15_rec + a20_rec + a25_rec) / 5

print("The average of the recall score: ", rec7)

Accuracy Scores
0.5813084112149532
0.5663551401869158
0.5545171339563862
0.5602803738317756
0.5607476635514018
The average of the accuracy score:  0.5646417445482865
*******************************
Precision Scores
0.32822976879842336
0.37434614518076165
0.336081498262007
0.34541717875340316
0.33950969396109265
The average of the precision score:  0.3447168569911375
*******************************
Recall Scores
0.7147409338934763
0.6749000434593655
0.6985780739453056
0.6983959872519194
0.694674682505191
The average of the recall score:  0.6962579442110516


### Selecting Eight Features

In [24]:
a5 = acc_test(8,5)
a10 = acc_test(8,10)
a15 = acc_test(8,15)
a20 = acc_test(8,20)
a25 = acc_test(8,25)

a5_acc = stat.mean(a5[0])
a10_acc = stat.mean(a10[0])
a15_acc = stat.mean(a15[0])
a20_acc = stat.mean(a20[0])
a25_acc = stat.mean(a25[0])

print("Accuracy Scores")
print(a5_acc)
print(a10_acc)
print(a15_acc)
print(a20_acc)
print(a25_acc)

acc8 = (a5_acc + a10_acc + a15_acc + a20_acc + a25_acc) / 5

print("The average of the accuracy score: ", acc8 )

print("*******************************")
print("Precision Scores")
a5_prec = stat.mean(a5[1])
a10_prec = stat.mean(a10[1])
a15_prec = stat.mean(a15[1])
a20_prec = stat.mean(a20[1])
a25_prec = stat.mean(a25[1])

print(a5_prec)
print(a10_prec)
print(a15_prec)
print(a20_prec)
print(a25_prec)

pre8 = (a5_prec + a10_prec + a15_prec + a20_prec + a25_prec) / 5

print("The average of the precision score: ", pre8 )

print("*******************************")
print("Recall Scores")
a5_rec = stat.mean(a5[2])
a10_rec = stat.mean(a10[2])
a15_rec = stat.mean(a15[2])
a20_rec = stat.mean(a20[2])
a25_rec = stat.mean(a25[2])

print(a5_rec)
print(a10_rec)
print(a15_rec)
print(a20_rec)
print(a25_rec)

rec8 = (a5_rec + a10_rec + a15_rec + a20_rec + a25_rec) / 5

print("The average of the recall score: ", rec8)

Accuracy Scores
0.5439252336448598
0.5485981308411214
0.5457943925233645
0.5499999999999999
0.5510280373831775
The average of the accuracy score:  0.5478691588785046
*******************************
Precision Scores
0.32523976296927115
0.3297644302593321
0.33201966815859435
0.32886229002712547
0.33738854832635246
The average of the precision score:  0.33065493994813505
*******************************
Recall Scores
0.700026075619296
0.696162055145106
0.7022106330580907
0.7026640591047371
0.7008022598870056
The average of the recall score:  0.7003730165628471


### Selecting Nine Features

In [25]:
a5 = acc_test(9,5)
a10 = acc_test(9,10)
a15 = acc_test(9,15)
a20 = acc_test(9,20)
a25 = acc_test(9,25)

a5_acc = stat.mean(a5[0])
a10_acc = stat.mean(a10[0])
a15_acc = stat.mean(a15[0])
a20_acc = stat.mean(a20[0])
a25_acc = stat.mean(a25[0])

print("Accuracy Scores")
print(a5_acc)
print(a10_acc)
print(a15_acc)
print(a20_acc)
print(a25_acc)

acc9 = (a5_acc + a10_acc + a15_acc + a20_acc + a25_acc) / 5

print("The average of the accuracy score: ", acc9 )

print("*******************************")
print("Precision Scores")
a5_prec = stat.mean(a5[1])
a10_prec = stat.mean(a10[1])
a15_prec = stat.mean(a15[1])
a20_prec = stat.mean(a20[1])
a25_prec = stat.mean(a25[1])

print(a5_prec)
print(a10_prec)
print(a15_prec)
print(a20_prec)
print(a25_prec)

pre9 = (a5_prec + a10_prec + a15_prec + a20_prec + a25_prec) / 5

print("The average of the precision score: ", pre9 )

print("*******************************")
print("Recall Scores")
a5_rec = stat.mean(a5[2])
a10_rec = stat.mean(a10[2])
a15_rec = stat.mean(a15[2])
a20_rec = stat.mean(a20[2])
a25_rec = stat.mean(a25[2])

print(a5_rec)
print(a10_rec)
print(a15_rec)
print(a20_rec)
print(a25_rec)

rec9 = (a5_rec + a10_rec + a15_rec + a20_rec + a25_rec) / 5

print("The average of the recall score: ", rec9)

Accuracy Scores
0.530841121495327
0.5327102803738317
0.540809968847352
0.5509345794392523
0.5416822429906542
The average of the accuracy score:  0.5393956386292834
*******************************
Precision Scores
0.30977103230132913
0.31498925102609343
0.3305686543184667
0.32578676819000457
0.32461392948595114
The average of the precision score:  0.321145927064369
*******************************
Recall Scores
0.7009667294413057
0.700667342701241
0.701194649669226
0.7046281809841132
0.7010347192042108
The average of the recall score:  0.7016983244000193


## Scorecard

### Accuracy Score

In [26]:
print("Accuracy Score")
print("Selection One Feature: ", acc1)
print("Selection Two Features: ", acc2)
print("Selection Three Features: ", acc3)
print("Selection Four Features: ", acc4)
print("Selection Five Features: ", acc5)
print("Selection Six Features: ", acc6)
print("Selection Seven Features: ", acc7)
print("Selection Eight Features: ", acc8)
print("Selection Nine Features: ", acc9)


Accuracy Score
Selection One Feature:  0.5657383177570093
Selection Two Features:  0.5355327102803737
Selection Three Features:  0.5829532710280374
Selection Four Features:  0.5757258566978194
Selection Five Features:  0.5674267912772585
Selection Six Features:  0.5694143302180684
Selection Seven Features:  0.5646417445482865
Selection Eight Features:  0.5478691588785046
Selection Nine Features:  0.5393956386292834


### Precision Score

In [27]:
print("Precision Score")
print("Selection One Feature: ", pre1)
print("Selection Two Features: ", pre2)
print("Selection Three Features: ", pre3)
print("Selection Four Features: ", pre4)
print("Selection Five Features: ", pre5)
print("Selection Six Features: ", pre6)
print("Selection Seven Features: ", pre7)
print("Selection Eight Features: ", pre8)
print("Selection Nine Features: ", pre9)

Precision Score
Selection One Feature:  0.4100975847227503
Selection Two Features:  0.3704093476258478
Selection Three Features:  0.3937679570461715
Selection Four Features:  0.3865799834477196
Selection Five Features:  0.3542900393154391
Selection Six Features:  0.35165598387287833
Selection Seven Features:  0.3447168569911375
Selection Eight Features:  0.33065493994813505
Selection Nine Features:  0.321145927064369


### Recall Score

In [28]:
print("Recall Score")
print("Selection One Feature: ", rec1)
print("Selection Two Features: ", rec2)
print("Selection Three Features: ", rec3)
print("Selection Four Features: ", rec4)
print("Selection Five Features: ", rec5)
print("Selection Six Features: ", rec6)
print("Selection Seven Features: ", rec7)
print("Selection Eight Features: ", rec8)
print("Selection Nine Features: ", rec9)

Recall Score
Selection One Feature:  0.6328107344632768
Selection Two Features:  0.654171983195712
Selection Three Features:  0.6742433289873968
Selection Four Features:  0.6785930175286108
Selection Five Features:  0.6924163020908785
Selection Six Features:  0.6970302525472016
Selection Seven Features:  0.6962579442110516
Selection Eight Features:  0.7003730165628471
Selection Nine Features:  0.7016983244000193


### Selecing the feature(s)

In [29]:
from sklearn.model_selection import train_test_split
med_SES = stat.median(ad_X_train['SES'])
avg_MMSE = ad_X_train['MMSE'].mean()
X_train, X_test, y_train, y_test = train_test_split(ad_X_train, ad_y_train, test_size=0.2, random_state=42)
X_train.loc[:,'SES'] = X_train.loc[:,'SES'].fillna(med_SES)
X_train.loc[:,'MMSE'] = X_train.loc[:,'MMSE'].fillna(avg_MMSE)

y_train = y_train.astype(float)
y_train.loc[:,'CDR'] = y_train.loc[:,'CDR'].fillna(stat.median(y_train['CDR']))
y_train = y_train.astype(str)

ridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(X_train,y_train)

sfs_forward = SequentialFeatureSelector(
ridge , n_features_to_select = 4, direction = "forward").fit(X_train,y_train)

X_test.columns[sfs_forward.get_support()]

Index(['MMSE', 'nWBV', 'ASF', 'Female'], dtype='object')